In [2]:
from kmodes.kprototypes import KPrototypes
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import numpy as np
from sklearn.preprocessing import MinMaxScaler


In [29]:
path = '/Users/mehmetdeniz/Library/CloudStorage/GoogleDrive-mehmetbakideniz@gmail.com/My Drive/meta_audience_insights_age_data.csv'
df=pd.read_csv(path)

In [36]:
def anomaly_detection_mixed_data_type(df, category_object, category_metric):
    aggregations = {'cost': 'sum', 'clicks': 'sum', 'purchase_value': 'sum', 'install': 'sum', 'impressions': 'sum','purchase': 'sum' }
    df2 = df.groupby(category_object).agg(aggregations).reset_index()

    df2['cpm'] = np.where(df2['impressions'] != 0, (df2['cost'] / df2['impressions'])*1000, 0)
    df2['cr'] = np.where(df2['clicks'] != 0, df2['purchase'] / df2['clicks'], 0)
    df2['cpi'] = np.where(df2['install'] != 0, df2['cost'] / df2['install'], 0)
    df2['ctr'] = np.where(df2['clicks'] != 0, df2['impressions'] / df2['clicks'], 0)
    df2['cpc'] = np.where(df2['clicks'] != 0, df2['cost'] / df2['clicks'], 0)
    df2['roas']=np.where(df2['cost'] != 0, df2['purchase_value'] / df2['cost'], 0)
    df2.drop(['clicks', 'purchase_value', 'cost', 'install', 'impressions','purchase'], axis=1, inplace= True )
    df2.reset_index(inplace=True, drop=True)
    categorical_columns = df2.select_dtypes(include=['object']).columns
    indexes = [df2.columns.get_loc(column) for column in categorical_columns]
    df_subset=df2[category_object+category_metric]
    numerical_columns = df_subset.select_dtypes(include=[float, int]).columns
    
    minmax_scaler = MinMaxScaler(feature_range=(-1, 1))
    
    df_subset[numerical_columns] = minmax_scaler.fit_transform(df_subset[numerical_columns])
    kp = KPrototypes(n_clusters=1, init='Huang', n_init=3, verbose=True)
    kp.fit_predict(df_subset, categorical=indexes)
    df_subset['labels']=kp.labels_
    clusters=kp.cluster_centroids_

    def calculate_distances(df_subset, centroids):
        def euclidean_dissim(a, b):
            """Euclidean distance dissimilarity function"""
            return np.sum((a - b) ** 2)
    
        def matching_dissim(a, b):
            """Simple matching dissimilarity function"""
            return np.sum(a != b)
    
        # Initialize an empty list to store distance scores
        distances = []
    
        # Iterate over each row in the DataFrame
        for index, row in df_subset.iterrows():
            # Get the corresponding centroid for the current row
            indx=row['labels']
            centroid = centroids[indx]
            row = row.iloc[:-1]
            categorical_values = []
            numerical_values = []
            for value in row:
                if isinstance(value, (str, pd.Categorical)):
                    categorical_values.append(value)
                else:
                    numerical_values.append(value)
    
            # Convert categorical and numerical sections to numpy arrays
            categorical_array = np.array(categorical_values)
            numerical_array = np.array(numerical_values)
    
            numeric_centroid = []
            categorical_centroid = []
    
            for value in centroid:
                try:
                    numeric_value = float(value)
                    numeric_centroid.append(numeric_value)
                except ValueError:
                    categorical_centroid.append(value)
    
            numeric_centroid = np.array(numeric_centroid)
            categoric_centroid = categorical_centroid
            categorical_dissimilarity = matching_dissim(categorical_array, categoric_centroid)
    
            numerical_dissimilarity = euclidean_dissim(numerical_array, numeric_centroid)
            gamma=kp.gamma
            distance = gamma*categorical_dissimilarity + numerical_dissimilarity
    
            distances.append(distance)
    
        df_subset['distance'] = distances
    
        return df_subset

    df_subset = calculate_distances(df_subset, clusters)
    df_subset[numerical_columns] = pd.DataFrame(minmax_scaler.inverse_transform(df_subset[numerical_columns]), columns=numerical_columns)
    df_subset.sort_values(by=['distance'], inplace=True, ascending=False)
    df_anomaly=df_subset
    # Calculate the number of rows for the first 5%
    num_rows = int(len(df_anomaly) * 0.2)

# Create the new DataFrame containing the first 5% of rows
    df_anomaly = df_anomaly.head(num_rows)
    df_anomaly.reset_index(drop=True, inplace=True)
    df_anomaly.drop(columns=['labels', 'distance'], inplace=True)
    return df_anomaly

In [39]:
a=['campaign_type','age']
b=['roas','cpc']
c=anomaly_detection_mixed_data_type(df,a,b)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/100, moves: 0, ncost: 26.52191688453656
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 2, iteration: 1/100, moves: 0, ncost: 26.52191688453656
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 3, iteration: 1/100, moves: 0, ncost: 26.52191688453656
Best run was number 1


/var/folders/9l/rh0wv2td1gn_vk4kl3gcqzn80000gn/T/ipykernel_4852/2725129410.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset[numerical_columns] = minmax_scaler.fit_transform(df_subset[numerical_columns])
/var/folders/9l/rh0wv2td1gn_vk4kl3gcqzn80000gn/T/ipykernel_4852/2725129410.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['labels']=kp.labels_
/var/folders/9l/rh0wv2td1gn_vk4kl3gcqzn80000gn/T/ipykernel_4852/2725129410.py:76: SettingWithCopyWarning: 
A value is trying to be set o

In [40]:
c

,campaign_type,age,roas,cpc
0,APP_INSTALLS,65+,28.004128,0.620065
1,LINK_CLICKS,55-64,0.000000,1.110918
2,LINK_CLICKS,18-24,10.950329,1.148742
3,LINK_CLICKS,45-54,8.639121,1.106779
